In [19]:
import os

In [20]:
%pwd

'e:\\ML_Projects'

In [21]:
os.chdir("../")

In [22]:
%pwd

'e:\\'

In [ ]:
# config entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
# configuration 
from src.wine.constants import *
from src.wine.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [ ]:

import os
import zipfile
import logging
import urllib.request as request
from src.wine import logger
from src.wine.utils.common import get_size
from pathlib import Path

# Assuming DataIngestionConfig is defined elsewhere
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.logger = logging.getLogger(__name__)  # Initialize logger

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            self.logger.info(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            self.logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the ZIP file specified in the config to the designated directory.
        """
        zip_file_path = self.config.local_data_file
        extract_to = self.config.unzip_dir
        
        try:
            # Check if the file exists
            if not os.path.exists(zip_file_path):
                raise ValueError(f"The file {zip_file_path} does not exist.")

            # Check if the file is a valid ZIP file
            if not zipfile.is_zipfile(zip_file_path):
                raise ValueError(f"The file {zip_file_path} is not a valid ZIP file.")

            # Create the extraction directory if it doesn't exist
            os.makedirs(extract_to, exist_ok=True)

            # Extract the ZIP file
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
            
            self.logger.info(f"Successfully extracted {zip_file_path} to {extract_to}")

        except Exception as e:
            self.logger.error(f"An error occurred while extracting the ZIP file: {e}")

# Example usage:
# config = DataIngestionConfig(source_URL='http://example.com/data.zip', local_data_file='data.zip', unzip_dir='extracted_data')
# data_ingestion = DataIngestion(config)
# data_ingestion.download_file()
# data_ingestion.extract_zip_file()


In [ ]:
# pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-17 03:47:35,482] INFO: common: yaml file: E:\ML_Projects\Wine_Prediction_Project\config\config.yaml loaded successfully]
[2024-11-17 03:47:35,489] INFO: common: yaml file: E:\ML_Projects\Wine_Prediction_Project\params.yaml loaded successfully]
[2024-11-17 03:47:35,493] INFO: common: yaml file: E:\ML_Projects\Wine_Prediction_Project\schema.yaml loaded successfully]
[2024-11-17 03:47:35,497] INFO: common: created directory at: artifacts]
[2024-11-17 03:47:35,501] INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-17 03:47:37,250] INFO: 2359738225: artifacts/data_ingestion/data.zip downloaded! with the following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny